In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Wide-Transformation").master("yarn").getOrCreate()

spark


25/06/29 22:26:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
customers_data = [
"customer_id,name,city,state,country,registration_date,is_active",
"0,Customer_0,Bangalore,Karnataka,India,2023-11-11,True",
"1,Customer_1,Hyderabad,Delhi,India,2023-08-26,True",
"2,Customer_2,Ahmedabad,West Bengal,India,2023-06-23,True",
"3,Customer_3,Bangalore,Tamil Nadu,India,2023-03-24,False",
"4,Customer_4,Bangalore,Gujarat,India,2023-06-06,False",
"5,Customer_5,Delhi,Maharashtra,India,2023-04-19,False"]

In [3]:
data_rdd = spark.sparkContext.parallelize(customers_data)

In [4]:
# Filter () -> Transformation
# first () -> Action

header = data_rdd.first()

data_rdd = data_rdd.filter(lambda row : row!=header)

In [5]:
def parse_data(row):
    fields = row.split(',')
    
    return (
        int(fields[0]),
        fields[1],
        fields[2],
        fields[3],
        fields[4],
        fields[5],
        fields[6]=='True'
    )

In [6]:
parsed_rdd = data_rdd.map(parse_data)

In [7]:
active_customers = parsed_rdd.filter(lambda row:row[6])

In [8]:
print(parsed_rdd.getNumPartitions())

2


In [9]:
grouped_by_city_rdd = parsed_rdd.map(lambda row: (row[2], 1)).reduceByKey(lambda x, y: x + y)

In [10]:
grouped_by_city_rdd.collect()

[('Hyderabad', 1), ('Delhi', 1), ('Ahmedabad', 1), ('Bangalore', 3)]

In [11]:
spark.stop()